In [ ]:
!python3 -m pip install syft

In [1]:
import syft as sy
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


/home/mg6827/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/mg6827/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


In [2]:
import pandas as pd
import torch
import numpy as np
import os
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from syft.util import get_root_data_path
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)

# duet1.store.pandas

in_dim = 4
out_dim = 3
n_samples = 150

class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv2d(3, 32, 3, 1)
        self.conv2 = self.torch_ref.nn.Conv2d(32, 64, 3, 1) 
        self.dropout = self.torch_ref.nn.Dropout2d(0.25)
        self.fc1 = self.torch_ref.nn.Linear(147456, 128)
        self.fc2 = self.torch_ref.nn.Linear(128, 4)

    def forward(self, x):
        x = self.torch_ref.nn.functional.relu(self.conv1(x))
        x = self.torch_ref.nn.functional.relu(self.conv2(x))
        x = self.torch_ref.nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = self.torch_ref.flatten(x, 1)
        x = self.torch_ref.nn.functional.relu(self.fc1(x))
        x = self.fc2(self.dropout(x))
        output = self.torch_ref.nn.functional.log_softmax(x, dim=1)
        return output


# def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):
#     print('training started')

#     losses = []

#     for i in range(iterations):
#         print('iteration: ', i)
#         optim.zero_grad()
#         print('zerograd done')
#         output = model(data_ptr)

#         # nll_loss = negative log-liklihood loss
#         loss = torch_ref.nn.functional.nll_loss(output, target_ptr.long())

#         loss_item = loss.item()

#         loss_value = loss_item.get(
#             reason="To evaluate training progress", request_block=True, timeout_secs=5
#         )

#         if i % 1 == 0:
#             print("Epoch", i, "loss", loss_value)

#         losses.append(loss_value)

#         loss.backward()

#         optim.step()

#     return losses

In [3]:
def train(epochs, model, torch_ref, optim, training_data_loader):
    print('training started')
    for epoch in range(epochs):

        model.train()
        epoch_loss = 0
        for batch_idx, data_pointers in enumerate(training_data_loader):

            optim.zero_grad()
            data_ptr, target_ptr = data_pointers[0], data_pointers[1]
            data_ptr_reshape = torch_ref.unsqueeze(data_ptr,0)
            target_ptr_reshape = torch_ref.unsqueeze(target_ptr, 0)

            output_ptr = model(data_ptr_reshape)
            loss = torch_ref.nn.functional.nll_loss(output_ptr, target_ptr_reshape)
            loss.backward()
            optim.step()
            loss_item = loss.item().get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=3,
                delete_obj=False,
                verbose=False )
            epoch_loss += loss_item
        print('loss', epoch, epoch_loss)
#             print(epoch, batch_idx, loss.item)

#             if batch_idx % 1 == 0:
#                 loss_item = loss.item().get(
#                     reason="To evaluate training progress",
#                     request_block=True,
#                     timeout_secs=3,
#                     delete_obj=False,
#                     verbose=False
#                     )
#                 print(f"epoch {epoch}, batch_idx {batch_idx}, loss {loss_item}")

In [4]:
data_ptr1 = duet1.store["x_train"]
target_ptr1 = duet1.store["y_train"]
train_num = duet1.store["train_num"]
# target_vals = target_ptr1.get(
#             request_block=True,
#             reason="To write the training loop",
#             timeout_secs=30,
#             delete_obj=False,
#             verbose=True )
# print('target_val', target_vals, target_vals.shape)
    
print(data_ptr1, target_ptr1, train_num)
import torch.utils.data as data
from torch.utils.data import DataLoader
class DatasetFromPointer(data.Dataset):
    def __init__(self, 
                 X_tensorpointer,
                 y_tensorpointer,
                 datanum_pointer,
                 ):
        super(DatasetFromPointer, self).__init__()
        self.X_tensorpointer = X_tensorpointer
        self.y_tensorpointer = y_tensorpointer
        self.datanum_pointer = datanum_pointer

    def __getitem__(self, index):
        input = self.X_tensorpointer[index]
        target = self.y_tensorpointer[index]
        return input, target

    def __len__(self):
        result = self.datanum_pointer.get(
        request_block=True,
        reason="To write the training loop",
        timeout_secs=30,
        delete_obj=False,
        verbose=True )
        print('result',result)
        return result
        
        
def batch_idx_fn(batch):
  return batch[0]

train_set = DatasetFromPointer(data_ptr1, target_ptr1, train_num)
training_data_loader = DataLoader(dataset=train_set, 
                                batch_size=4, shuffle=True,
                                  collate_fn=batch_idx_fn)

# num_workers=4,

<syft.proxy.torch.TensorPointer object at 0x151e3acec6d0> <syft.proxy.torch.TensorPointer object at 0x151e3acec070> <syft.proxy.syft.lib.python.IntPointer object at 0x151e3acecac0>
result 1897
result 1897


In [5]:
base_model = SyNet(torch)
remote_model1 = base_model.send(duet1)
remote_torch1 = duet1.torch

args = {
    "batch_size": 64,
    "test_batch_size": 1000,
    "epochs": 14,
    "lr": 1.0,
    "gamma": 0.7,
    "no_cuda": False,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
}

In [6]:
# lets ask to see if our Data Owner has CUDA
has_cuda = False
has_cuda_ptr = remote_torch1.cuda.is_available()
print('hashsahs', has_cuda_ptr)
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    reason="To run test and inference locally",
    timeout_secs=5,  # change to something slower
))
print(has_cuda)

use_cuda = not args["no_cuda"] and has_cuda
# use_cuda = has_cuda
# now we can set the seed
remote_torch1.manual_seed(args["seed"])

device = remote_torch1.device("cuda" if use_cuda else "cpu")
print(f"Data Owner device is {device.type.get()}")

# if we have CUDA lets send our model to the GPU
if has_cuda:
    remote_model1.cuda(device)
else:
    remote_model1.cpu()


hashsahs <syft.proxy.syft.lib.python.BoolPointer object at 0x151e3ac6e2b0>
False
Data Owner device is cpu


In [ ]:
params = remote_model1.parameters()
optim = remote_torch1.optim.Adam(params=params, lr=0.01)
print("params:", params)
print("optim:", optim)

epochs = 20
losses = train(epochs, remote_model1, remote_torch1, optim, training_data_loader)

params: <syft.proxy.syft.lib.python.ListPointer object at 0x151e3ac6e880>
optim: <syft.proxy.torch.optim.AdamPointer object at 0x151e3ac6e5e0>
training started
result 1897
loss 0 1671.1908284680612
result 1897


In [7]:
for batch_idx, data_pointers in enumerate(training_data_loader):
    print(batch_idx, data_pointers)

result 31
0 (<syft.proxy.torch.TensorPointer object at 0x15533dbe8fa0>, <syft.proxy.torch.TensorPointer object at 0x15533e787ac0>)
1 (<syft.proxy.torch.TensorPointer object at 0x15533e787910>, <syft.proxy.torch.TensorPointer object at 0x15533e787bb0>)
2 (<syft.proxy.torch.TensorPointer object at 0x15533dbe8eb0>, <syft.proxy.torch.TensorPointer object at 0x15533dbe8c40>)
3 (<syft.proxy.torch.TensorPointer object at 0x15533e787370>, <syft.proxy.torch.TensorPointer object at 0x1553e74e1250>)
4 (<syft.proxy.torch.TensorPointer object at 0x15533e787ac0>, <syft.proxy.torch.TensorPointer object at 0x15533f31c0d0>)
5 (<syft.proxy.torch.TensorPointer object at 0x15533e787b50>, <syft.proxy.torch.TensorPointer object at 0x15533e787310>)
6 (<syft.proxy.torch.TensorPointer object at 0x15533e787730>, <syft.proxy.torch.TensorPointer object at 0x1553e74e1250>)
7 (<syft.proxy.torch.TensorPointer object at 0x15533e7878b0>, <syft.proxy.torch.TensorPointer object at 0x15533dbe8eb0>)
